In [3]:
import pickle
import os
import matplotlib.pyplot as plt
from imageio import imread
import math
import numpy as np
import pandas as pd
import math
import logging
import downloader
import recommender as rec
import json

# TODO: remove global...
CM_TO_KM = 100000           # CM in a KM
MAX_MAP_SIZE = 8            # In KM


# search for specific attributes
# player_id :   players acount id (eg. 'acount.*****')
# start_range : starting time (eg. 'YYYY-MM-DDTHH:MM:SS.SSSZ')
# end_range:    ending time
# event_type:   list of type of event (eg. ['LogParachutepiLanding'])
def search(json_object, player_id=None, start_range=None, end_range=None, event_type=None):
    events = []
    i = 0
    for entry in json_object:
        if (is_event(entry, event_type) and
                in_range(entry['_D'], start_range, end_range) and
                has_player(entry, player_id)):

            events.append(entry)
    return events


def in_range(time, start, end):
    if start is None and end is None:
        return True
    elif start is None:
        return is_after(end, time)
    elif end is None:
        return is_after(time, start)
    else:
        return is_after(time, start) and is_after(end, time)


def is_after(time1, time2):
    t_index = time1.find('T')
    date1 = time1[:t_index].encode('ascii', 'ignore')
    date1 = date1.split('-')
    time1 = time1[t_index + 1:][:-1].encode('ascii', 'ignore')
    time1 = time1.split(':')

    t_index = time2.find('T')
    date2 = time2[:t_index].encode('ascii', 'ignore')
    date2 = date2.split('-')
    time2 = time2[t_index + 1:][:-1].encode('ascii', 'ignore')
    time2 = time2.split(':')

    equals = True

    for x in range(3):
        if int(date1[x]) > int(date2[x]):
            return True
    for x in range(2):
        if int(time1[x]) > int(time2[x]):
            return True

    if float(time1[2]) > float(time2[2]):
        return True

    if time1 != time2:
        return False

    return equals


def has_player(event, player_id):
    if player_id is None:
        return True

    for key in ['character', 'attacker', 'victim', 'assistant', 'reviver']:
        if key in event.keys():
            if event[key]['accountId'] == player_id:
                return True
        else:
            return False
    return False


def is_event(event, event_type):
    if event_type is None:
        return True
    else:
        return event['_T'] in event_type


# Return the dict from the pickle file name
def load_pickle(pickle_file):
    with open(pickle_file, 'rb') as f:
        telemetry_data = pickle.load(f)
    f.close()
    return telemetry_data


# Get the location the plane starts in (not quite sure it's useful yet...)
def get_plane_start(telemetry):
    start_loc = None
    for log_entry in telemetry:
        if log_entry["_T"] == "LogMatchStart":
            # All players start at the exact same location, so we only need first
            start_loc = log_entry["characters"][0]['location']
    return start_loc


# Get the map name from telemetry
def get_map(telemetry):
    for log_entry in telemetry:
        if log_entry["_T"] == "LogMatchStart":
            return log_entry["mapName"]
    return None


# Get the result for each player in the match from the telemetry data
# Return the results as a list of dicts:
#       [{'name' : playerName, 'ranking': ranking},
#        ...]
def get_rankings(telemetry):
    results = []
    for log_entry in telemetry:
        if log_entry["_T"] == "LogMatchEnd":
            for character in log_entry['characters']:
                results.append({'name': character['name'],
                                'ranking': character['ranking']})
    return results


# Get the location of each player's landing from the telemetry data
# Return as a dict:
#           {playerName: [x, y],
#            ...}
def get_all_landings(telemetry):
    landings = {}
    for log_entry in telemetry:
        if log_entry["_T"] == "LogParachuteLanding":
            landing_loc = log_entry['character']['location']
            x = landing_loc['x']
            y = landing_loc['y']
            landings[log_entry['character']['name']] = [x, y]
    return landings


# Returns the first  and last locations someone jumped out of the plane
def get_flight_data(telemetry):
    first_coordinate = None # First player exit event from plane
    current_coordinate = None # Last player exist even from plane
    for log_entry in telemetry:
        if log_entry.get("_T") == "LogVehicleLeave" \
                and log_entry.get("vehicle").get("vehicleId") == "DummyTransportAircraft_C":
            current_coordinate = log_entry.get("character").get("location")
            if first_coordinate is None:
                first_coordinate = current_coordinate
    return first_coordinate, current_coordinate


# Returns the unit vector for the flight path given the first and last location someone exited plane
def get_flight_vector(first_drop, last_drop):
    flight_vector = np.array([last_drop['x'] - first_drop['x'], last_drop['y'] - first_drop['y']])
    vector_length = np.linalg.norm(flight_vector)
    flight_vector_norm = flight_vector / vector_length
    return flight_vector_norm


# Returns the angle between the two flight vectors u and v (squared)
# Note: u and v must be unit vectors already
def flight_diff(u, v):
    return np.arccos(np.clip(np.dot(u, v), -1, 1))**2


# Given a flight path (unit vectorized), get the direction that it is closest to
# from one of the eight recognized cardinal directions
def get_flight_category(flight_vector):
    # Set up unit vectors for the 8 directions to classify flight paths as
    between_mag = 1 / math.sqrt(2)
    nn = [0, 1]
    ne = [between_mag, between_mag]
    ee = [1, 0]
    se = [between_mag, -between_mag]
    ss = [0, -1]
    sw = [-between_mag, -between_mag]
    ww = [-1, 0]
    nw = [-between_mag, between_mag]

    # Put em in a DataFrame for processing
    directions = np.array([nn, ne, ee, se, ss, sw, ww, nw])
    dirs_df = pd.DataFrame(directions)

    # Combine the x, y component columns, add direction names, and get rid of the excess
    dirs_df['direction_vec'] = list(zip(dirs_df[0], dirs_df[1]))
    dirs_df['direction'] = np.array(['nn', 'ne', 'ee', 'se', 'ss', 'sw', 'ww', 'nw'])
    dirs_df.drop([0, 1], axis=1, inplace=True)

    # Calculate the angle between the flight_vector and each of the 8 directions
    # and store it under a new column, 'angle_from_path'
    dirs_df['angle_from_path'] = dirs_df['direction_vec'].apply(flight_diff, args=(flight_vector,))

    # And return the direction (string) where the angle between the flight_vector
    # and the direction is minimized (i.e. the closest direction to the flight_vector)
    return dirs_df['direction'].loc[dirs_df['angle_from_path'].idxmin()]


# Convert raw (x, y) coordinates to the category
# Maps are divided into a 20x20 grid of square_size x square_size blocks
# where each square can be represented by a letter for the x and y position of that
# square in the map, e.g. AA for the square containing (0, 0)
def get_loc_cat(x, y, map_dim):
    square_size = 0.05 * map_dim
    x_ = x // square_size
    y_ = y // square_size

    # add the number square along each access to 'A's ascii code and return the characters
    # as the category
    return chr(65 + int(x_)) + chr(65 + int(y_))


# Plot the drop locations of each player (in blue), with opacity in relation to their rank in that match
# (more opaque = lower rank), along with the location the first person left the plane (in green)
# and the last person to leave the plane (in red)
def display_drop_locations(telemetry, fig, fig_x, fig_y, fig_num, match_num):
    landings = get_all_landings(telemetry)
    rankings = get_rankings(telemetry)
    map_name = get_map(telemetry)

    # Set up plot scale
    if map_name == "Savage_Main":                        # 4km map
        x_max = MAX_MAP_SIZE * (1/2)
        y_max = x_max
        map_img = imread("savage.png")
        plt.imshow(map_img, zorder=0, extent=[0.0, 4.0, 0.0, 4.0])
    elif map_name == "Erangel_Main":                     # 8km map
        x_max = MAX_MAP_SIZE
        y_max = MAX_MAP_SIZE
        map_img = imread("erangel.png")
        plt.imshow(map_img, zorder=0, extent=[0.0, 8.0, 0.0, 8.0])
    elif map_name ==  "Desert_Main":                     # 8km map
        x_max = MAX_MAP_SIZE
        y_max = x_max
        map_img = imread("miramar.png")
        plt.imshow(map_img, zorder=0, extent=[0.0, 8.0, 0.0, 8.0])
    elif map_name == "DihorOtok_Main":                   # 6km map
        x_max = MAX_MAP_SIZE * (3/4)
        y_max = x_max
        map_img = imread("vikendi.png")
        plt.imshow(map_img, zorder=0, extent=[0.0, 6.0, 0.0, 6.0])

    first_launch, last_launch = get_flight_data(telemetry)

    if first_launch is not None:
        launch_x = [first_launch['x'], last_launch['x']]
        launch_y = [first_launch['y'], last_launch['y']]

        ax = fig.add_subplot(fig_x, fig_y, fig_num)

        # plot first and last jump locations
        ax.scatter(launch_x[0] / CM_TO_KM, launch_y[0] / CM_TO_KM, s=100,
                   color='green', edgecolors='black', zorder=1)
        ax.scatter(launch_x[1] / CM_TO_KM, launch_y[1] / CM_TO_KM, s=100,
                   color='red', edgecolors='black', zorder=1)

        # plot line between them
        ax.plot([x_ / CM_TO_KM for x_ in launch_x],
                [y_ / CM_TO_KM for y_ in launch_y], 'grey', linestyle='--', marker='', zorder=1)

        # plot each player according to their ranking
        for ranking in rankings:
            landing_loc = landings[ranking['name']]
            # print("Player {} landing at position\t ({}, {}) and ended up rank : {}".format(ranking['name'],
            #                                                                           landing_loc[0],
            #                                                                          landing_loc[1],
            #                                                                          ranking['ranking']))
            if ranking['ranking'] == 1:
                ax.scatter(landing_loc[0] / CM_TO_KM, landing_loc[1] / CM_TO_KM,
                           color='yellow', edgecolors='black', zorder=1)
            else:
                ax.scatter(landing_loc[0] / CM_TO_KM, landing_loc[1] / CM_TO_KM,
                           color='blue', alpha=1/ranking['ranking'], zorder=1)
        plt.ylim(0, y_max)
        plt.xlim(0, x_max)
        plt.xlabel('km')
        plt.ylabel('km')
        plt.title(map_name)
        plt.savefig('./match_landings/match_{}.png'.format(match_num))
        plt.show()
    else:
        logging.error("Could not get launch data")


def build_drop_data(telemetry_files):
    # data_dir = "./data/"
    data_dir = "D:/Matches/"
    drop_data = []

    # Plots each match landing locations on a new plot
    for match_num in range(0, len(telemetry_files)):
        logging.debug("Building match %i of %i", match_num, len(telemetry_files) - 1)
        # If the filesize is greater than 0, ie there is actual data in it
        if os.path.getsize(data_dir + telemetry_files[match_num]) > 0:
            try:
                telemetry = load_pickle(data_dir + telemetry_files[match_num])
            except EOFError:
                logging.error("Match file terminated unexpectedly, skipping")
                continue  # Skip processing files that terminate early
            first, last = get_flight_data(telemetry)
            map_name = get_map(telemetry)

            # Set map_size (in cm, like player locations)
            if map_name == "Savage_Main":  # 4km map
                map_size = 400000
            elif map_name == "Erangel_Main":  # 8km map
                map_size = 800000
            elif map_name == "Desert_Main":  # 8km map
                map_size = 800000
            elif map_name == "DihorOtok_Main":  # 6km map
                map_size = 600000

            # Get the flight direction
            if first is not None:
                flight_vec = get_flight_vector(first, last)
                dir = get_flight_category(flight_vec)

            # Get the landings
            landing_locs = get_all_landings(telemetry)
            rankings = get_rankings(telemetry)

            for player, loc in landing_locs.items():
                # For each player who dropped, get their rank from the rankings array
                loc_category = get_loc_cat(loc[0], loc[1], map_size)
                for ranking in rankings:
                    if player in ranking.values():
                        player_rank = ranking['ranking']

                drop_data.append({'player': player,
                                  'drop_loc_raw': loc,
                                  'drop_loc_cat': loc_category,
                                  'rank': player_rank,
                                  'flight_path': dir,
                                  'map': map_name})
                #print("{} ==> {}".format(flight_vec, dir))
            #display_drop_locations(telemetry, plt.figure(), 1, 1, 1, match_num)

    return pd.DataFrame(drop_data)


# Get safe zone and poison zone states (location and radius) throughout the game
# Returns dataframe for each time & states
# columns: ['_D', 'safetyZonePosition_x', 'safetyZonePosition_y', 'safetyZoneRadius', ...]
def getZoneStates(json_object):
    logGameStates = search(json_object, None, None, None, ['LogGameStatePeriodic'])
    allStates = []
    for gameState in logGameStates:
        timestamp = gameState['_D']
        state = gameState['gameState']
        newStateObj = {k : state[k] for k in ('safetyZoneRadius',
                                              'poisonGasWarningRadius')}
        safePos = state['safetyZonePosition']
        poisPos = state['poisonGasWarningPosition']

        newStateObj['safetyZonePosition_x'] = safePos['x']
        newStateObj['safetyZonePosition_y'] = safePos['y']
        newStateObj['_D'] = timestamp
        newStateObj['poisonGasWarningPosition_x'] = poisPos['x']
        newStateObj['poisonGasWarningPosition_y'] = poisPos['y']
        allStates.append(newStateObj)
    df = pd.DataFrame(allStates)
    return df

# Get items picked up (house, crate, loot, etc) and by whom
# Returns dataframe for each pickup log
# columns: ['character_accountId', 'character_name', 'item_category', ...]
def getItemPickup(json_object):
    itemPicks = search(json_object, 
                        None, 
                        None, 
                        None,   
                        ['LogItemPickup', 
                                'LogItemPickupFromCarepackage', 
                                'LogItemPickupFromLootBox'])
    parsed_data = []
    for log in itemPicks:
        char = log['character']
        item = log['item']
        pickupState = {
                    '_D' : log['_D'],
                    'character_accountId' : char['accountId'],
                    'character_name' : char['name'],
                    'item_category' : item['category'], 
                    'item_subCategory' : item['subCategory'],
                    'item_Id' : item['itemId']}
        parsed_data.append(pickupState)
    return pd.DataFrame(parsed_data)

# Returns a list of DataFrames where each DataFrame contains all of the drop data for a given map and flight path
def get_drop_data():
    # data_dir = "./data/"
    data_dir = "D:/Matches/"
    match_files = []
    telemetry_files = []

    downloader.setup_logging(True)
    logging.info("Scanning for match and telemetry files in %s to parse", data_dir)
    for file in os.listdir(data_dir):
        if "_match" in file:
            logging.debug("Match file %s found, adding as match", file)
            match_files.append(file)
        elif "_telemetry" in file:
            logging.debug("Telemetry file %s found, adding as match", file)
            telemetry_files.append(file)

    # Get aggregate data
    drop_data = build_drop_data(telemetry_files)

    # Split by map and flight path
    all_data = []
    map_data_li = split_drop_data_by_map(drop_data)
    for map_df in map_data_li:
        if map_df['map'] != "Savage_Main":
            flight_data_li = split_drop_data_by_flight_path(map_df)
        else:
            flight_data_li = map_df
        all_data.extend(flight_data_li)

    return all_data


# Split the DataFrame containing all of the drop data into separate DataFrames for each map
def split_drop_data_by_map(drop_data):
    map_data = []
    for map in drop_data['map'].unique():
        map_data.append(drop_data[drop_data['map'] == map])
    return map_data

# Split the drop data (assumed to already be split by map) by flight path
def split_drop_data_by_flight_path(drop_data):
    flight_data = []
    print(drop_data.columns)
    for flight in drop_data['flight_path'].unique():
        flight_data.append(drop_data[drop_data['flight_path'] == flight])
    print(len(flight_data))
    return flight_data

drop_data = get_drop_data()

for df in drop_data:
    print("\n", df.iloc[0][['map', 'flight_path']], " - ", df.shape[0], )
"""
map_savage_data = rec.preprocess_data(drop_data[drop_data['map'] == "Savage_Main"])
map_erangel_data = rec.preprocess_data(drop_data[drop_data["map"] == "Erangel_Main"])
map_desert_data = rec.preprocess_data(drop_data[drop_data['map'] == 'Desert_Main'])
"""
max_k = 20              # training model hyperparam, anything above this doesn't tell us much


print("######PRINTING RESULTS FOR DROP LOCATION PREDICTIONS##########\n\n")
#rec.train_model(drop_data[0], max_k)
print("PRINTING SAVAGE_MAIN RESULTS: ")
#rec.train_model(map_savage_data, max_k)
print("PRINTING ERANGEL_MAIN RESULTS: ")
#rec.train_model(map_erangel_data, max_k)
print("PRINTING DESERT_MAIN RESULTS: ")
#rec.train_model(map_desert_data, max_k)

print("###########DONE PRINTING DROP LOCATIONS PREDICTIONS###########\n\n")

DEBUG:root:Setting up logging


2019-06-04 12:07:12,469 - root - DEBUG - Setting up logging
2019-06-04 12:07:12,469 - root - DEBUG - Setting up logging


INFO:root:Scanning for match and telemetry files in D:/Matches/ to parse


2019-06-04 12:07:12,477 - root - INFO - Scanning for match and telemetry files in D:/Matches/ to parse
2019-06-04 12:07:12,477 - root - INFO - Scanning for match and telemetry files in D:/Matches/ to parse
2019-06-04 12:07:12,477 - root - INFO - Scanning for match and telemetry files in D:/Matches/ to parse


DEBUG:root:Telemetry file 0c2f95af-2dc3-4f08-876f-34789c8357d2_telemetry.pickle found, adding as match


...

DEBUG:root:Building match 97 of 658


2019-06-04 12:08:10,592 - root - DEBUG - Building match 97 of 658
2019-06-04 12:08:10,592 - root - DEBUG - Building match 97 of 658
2019-06-04 12:08:10,592 - root - DEBUG - Building match 97 of 658


DEBUG:root:Building match 98 of 658


2019-06-04 12:08:11,264 - root - DEBUG - Building match 98 of 658
2019-06-04 12:08:11,264 - root - DEBUG - Building match 98 of 658
2019-06-04 12:08:11,264 - root - DEBUG - Building match 98 of 658


KeyboardInterrupt: 

In [78]:
for i in range(len(drop_data)):
    df = drop_data[i]
    df = df[df["rank"] <= 10]
    print(df)
    df['x_drop_loc_raw'] = df['drop_loc_raw'].apply(lambda x: x[0])
    df['y_drop_loc_raw'] = df['drop_loc_raw'].apply(lambda x: x[1])
    drop_data[i] = df
    
# def isWinner(row):
#     if row['rank'] <= 5:
#         return "Winner"
#     else:
#         return "Loser"

       drop_loc_cat                      drop_loc_raw flight_path  \
0                QH     [339081.09375, 152563.390625]          nw   
2                RI        [349186.90625, 160740.125]          nw   
3                RH     [345526.09375, 154371.609375]          nw   
4                SH     [379695.09375, 149027.328125]          nw   
6                SG       [366499.625, 120417.921875]          nw   
11               SF     [363794.1875, 114268.4296875]          nw   
13               RH      [343474.40625, 156387.34375]          nw   
30               KJ         [202218.90625, 197842.25]          nw   
33               JK        [194840.25, 202073.265625]          nw   
38               JK    [197740.390625, 205884.921875]          nw   
42               JK           [199541, 200536.578125]          nw   
45               QM        [326790.34375, 243256.375]          nw   
46               GF     [133686.359375, 109823.03125]          nw   
51               GF  [128747.10156

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


       drop_loc_cat                     drop_loc_raw flight_path          map  \
95               SG    [368182.3125, 122332.0390625]          ww  Savage_Main   
97               SG       [372977.78125, 134492.375]          ww  Savage_Main   
105              RI       [355367.0625, 168236.6875]          ww  Savage_Main   
107              RI     [349101.8125, 172496.390625]          ww  Savage_Main   
110              SF      [362028.3125, 116455.03125]          ww  Savage_Main   
113              SF      [361848.59375, 116629.0625]          ww  Savage_Main   
126              MG  [245117.609375, 126313.8515625]          ww  Savage_Main   
131              MG  [246065.484375, 126788.3046875]          ww  Savage_Main   
133              RO        [350007.875, 284809.5625]          ww  Savage_Main   
134              RO         [348756.375, 281315.625]          ww  Savage_Main   
158              JG   [185840.359375, 136119.484375]          ww  Savage_Main   
162              JG        [

       drop_loc_cat                     drop_loc_raw flight_path          map  \
296              GN    [120680.546875, 260394.71875]          se  Savage_Main   
299              FM  [119889.0859375, 257661.734375]          se  Savage_Main   
300              GM  [122906.9296875, 259596.578125]          se  Savage_Main   
305              JK     [189380.40625, 204101.21875]          se  Savage_Main   
306              JK   [194728.421875, 203556.921875]          se  Savage_Main   
307              GM    [121520.5703125, 255489.6875]          se  Savage_Main   
309              JK     [195331.4375, 204889.765625]          se  Savage_Main   
311              JK    [194441.96875, 202719.328125]          se  Savage_Main   
331              JN     [181977.34375, 269122.15625]          se  Savage_Main   
334              JN     [186361.671875, 262202.1875]          se  Savage_Main   
335              JN      [183184.359375, 274093.375]          se  Savage_Main   
342              KE   [21991

       drop_loc_cat                     drop_loc_raw flight_path          map  \
578              HS    [150323.328125, 362768.40625]          ss  Savage_Main   
579              GS   [123290.6015625, 366526.34375]          ss  Savage_Main   
585              LR    [239027.015625, 348363.34375]          ss  Savage_Main   
589              MR    [240904.140625, 344728.53125]          ss  Savage_Main   
599              FQ    [114582.0390625, 329414.8125]          ss  Savage_Main   
601              OS     [295502.71875, 366484.34375]          ss  Savage_Main   
606              NN      [275241.59375, 272463.0625]          ss  Savage_Main   
607              NN      [272396.5625, 264995.78125]          ss  Savage_Main   
625              FS   [115976.7109375, 363331.96875]          ss  Savage_Main   
628              FP     [100567.7734375, 315985.125]          ss  Savage_Main   
634              JJ      [189290.4375, 190480.59375]          ss  Savage_Main   
636              JK     [190

...

       drop_loc_cat                    drop_loc_raw flight_path          map  \
5823             KO     [421515.28125, 569221.0625]          ss  Desert_Main   
5830             KN     [422406.65625, 522699.4375]          ss  Desert_Main   
5835             LN      [475354.3125, 520116.4375]          ss  Desert_Main   
5837             LM     [478644.71875, 517080.0625]          ss  Desert_Main   
5840             JL        [383002.25, 445195.6875]          ss  Desert_Main   
5846             JK          [374363, 436276.59375]          ss  Desert_Main   
5853             JK          [373478.28125, 428211]          ss  Desert_Main   
5859             LM       [444462.25, 516028.03125]          ss  Desert_Main   
5866             LN     [457580.84375, 524340.0625]          ss  Desert_Main   
5868             IJ      [321181.4375, 361366.6875]          ss  Desert_Main   
5870             II       [327996.71875, 357287.75]          ss  Desert_Main   
5872             HI    [312875.34375, 35

       drop_loc_cat                     drop_loc_raw flight_path  \
2744             MP     [383372.09375, 463796.84375]          se   
2751             JK        [291103.6875, 306512.625]          se   
2752             NQ       [401445.3125, 483000.1875]          se   
2755             MJ          [380006.5, 281864.0625]          se   
2761             MJ      [377560.46875, 284808.6875]          se   
2768             JK      [286897.96875, 307370.9375]          se   
2773             OK      [437189.4375, 313700.59375]          se   
2777             IH     [261182.65625, 236778.40625]          se   
2782             OK     [437432.34375, 311106.84375]          se   
2785             JH    [283469.84375, 220115.609375]          se   
2788             QL      [497285.4375, 346852.53125]          se   
2791             QL     [499423.78125, 346913.84375]          se   
2793             IH   [256358.421875, 233070.328125]          se   
2805             NC     [393582.3125, 89720.4921

       drop_loc_cat                    drop_loc_raw flight_path  \
2911             LQ       [334892.40625, 486848.25]          ss   
2917             MO     [373416.1875, 438010.96875]          ss   
2924             JN    [286666.46875, 415309.59375]          ss   
2925             JO     [271548.1875, 424632.09375]          ss   
2933             MJ          [375602.65625, 283570]          ss   
2938             MJ     [378822.4375, 284078.09375]          ss   
2947             HQ    [227704.765625, 488890.5625]          ss   
2957             PK     [454122.0625, 315606.78125]          ss   
2958             GK     [195791.40625, 313223.3125]          ss   
2960             OJ      [442897.125, 279470.84375]          ss   
2962             HQ   [225121.234375, 481680.59375]          ss   
2970             OK    [438130.03125, 310635.84375]          ss   
2974             GK       [194649.75, 317733.15625]          ss   
2977             NK    [409294.96875, 307399.71875]          s

       drop_loc_cat                    drop_loc_raw flight_path  \
12686            PE   [479090.59375, 129652.578125]          nn   
12687            PF       [450409.65625, 156320.75]          nn   
12688            PF        [456401.5625, 165508.75]          nn   
12689            PE    [474149.6875, 127062.890625]          nn   
12692            LG     [344782.5625, 196868.03125]          nn   
12693            OF    [446911.8125, 166816.796875]          nn   
12696            MJ     [379561.6875, 281881.53125]          nn   
12697            MJ      [382903.53125, 274961.125]          nn   
12703            MJ     [384723.15625, 274530.4375]          nn   
12708            JG        [299857.15625, 189537.5]          nn   
12709            PG     [475060.375, 183303.890625]          nn   
12710            QE   [483419.15625, 128023.671875]          nn   
12733            IK      [254416.375, 314659.28125]          nn   
12734            IK     [240856.8125, 318015.21875]          n

       drop_loc_cat                    drop_loc_raw flight_path  \
24564            OO    [428633.53125, 438987.03125]          sw   
24565            LQ           [334955.3125, 487300]          sw   
24566            OO       [428132.75, 440498.90625]          sw   
24567            LQ           [334530, 486099.6875]          sw   
24568            LQ         [330269.625, 483271.75]          sw   
24569            OP      [434853.1875, 450121.9375]          sw   
24573            LM    [358514.09375, 360373.28125]          sw   
24575            KR                [317051, 513140]          sw   
24576            LL     [357983.6875, 358262.34375]          sw   
24579            MO       [366837.25, 435244.15625]          sw   
24581            JQ      [291889.46875, 508847.875]          sw   
24586            JK      [283243.6875, 309454.0625]          sw   
24591            JK    [283526.46875, 303682.84375]          sw   
24592            LO     [340505.03125, 431955.4375]          s

In [79]:
for i in range(len(drop_data)):
    df = drop_data[i]
    df = df.drop(columns= ['drop_loc_raw'])
drop_data

[       drop_loc_cat                      drop_loc_raw flight_path  \
 0                QH     [339081.09375, 152563.390625]          nw   
 2                RI        [349186.90625, 160740.125]          nw   
 3                RH     [345526.09375, 154371.609375]          nw   
 4                SH     [379695.09375, 149027.328125]          nw   
 6                SG       [366499.625, 120417.921875]          nw   
 11               SF     [363794.1875, 114268.4296875]          nw   
 13               RH      [343474.40625, 156387.34375]          nw   
 30               KJ         [202218.90625, 197842.25]          nw   
 33               JK        [194840.25, 202073.265625]          nw   
 38               JK    [197740.390625, 205884.921875]          nw   
 42               JK           [199541, 200536.578125]          nw   
 45               QM        [326790.34375, 243256.375]          nw   
 46               GF     [133686.359375, 109823.03125]          nw   
 51               GF

In [80]:
drop_data[0]

,drop_loc_cat,drop_loc_raw,flight_path,map,player,rank,x_drop_loc_raw,y_drop_loc_raw
0,QH,"[339081.09375, 152563.390625]",nw,Savage_Main,Hisen28,3,339081.093750,152563.390625
2,RI,"[349186.90625, 160740.125]",nw,Savage_Main,Lee-Poet,3,349186.906250,160740.125000
3,RH,"[345526.09375, 154371.609375]",nw,Savage_Main,Cameronww,3,345526.093750,154371.609375
4,SH,"[379695.09375, 149027.328125]",nw,Savage_Main,ToveloXxInJ,2,379695.093750,149027.328125
6,SG,"[366499.625, 120417.921875]",nw,Savage_Main,helloyaji,2,366499.625000,120417.921875
11,SF,"[363794.1875, 114268.4296875]",nw,Savage_Main,MR2right,2,363794.187500,114268.429688
13,RH,"[343474.40625, 156387.34375]",nw,Savage_Main,alwaysthing,3,343474.406250,156387.343750
30,KJ,"[202218.90625, 197842.25]",nw,Savage_Main,ALeenyou,9,202218.906250,197842.250000
33,JK,"[194840.25, 202073.265625]",nw,Savage_Main,Forrest0320,7,194840.250000,202073.265625
38,JK,"[197740.390625, 205884.921875]",nw,Savage_Main,Tiramisu_0810,7,197740.390625,205884.921875


In [81]:
def success_category(x):
    return x // 20

for i in range(len(drop_data)):
    df = drop_data[i]
    df['success_category'] = df['rank'].apply(success_category)
    drop_data[i] = df

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [82]:
drop_data[0]

,drop_loc_cat,drop_loc_raw,flight_path,map,player,rank,x_drop_loc_raw,y_drop_loc_raw,isWinner
0,QH,"[339081.09375, 152563.390625]",nw,Savage_Main,Hisen28,3,339081.093750,152563.390625,True
2,RI,"[349186.90625, 160740.125]",nw,Savage_Main,Lee-Poet,3,349186.906250,160740.125000,True
3,RH,"[345526.09375, 154371.609375]",nw,Savage_Main,Cameronww,3,345526.093750,154371.609375,True
4,SH,"[379695.09375, 149027.328125]",nw,Savage_Main,ToveloXxInJ,2,379695.093750,149027.328125,True
6,SG,"[366499.625, 120417.921875]",nw,Savage_Main,helloyaji,2,366499.625000,120417.921875,True
11,SF,"[363794.1875, 114268.4296875]",nw,Savage_Main,MR2right,2,363794.187500,114268.429688,True
13,RH,"[343474.40625, 156387.34375]",nw,Savage_Main,alwaysthing,3,343474.406250,156387.343750,True
30,KJ,"[202218.90625, 197842.25]",nw,Savage_Main,ALeenyou,9,202218.906250,197842.250000,False
33,JK,"[194840.25, 202073.265625]",nw,Savage_Main,Forrest0320,7,194840.250000,202073.265625,False
38,JK,"[197740.390625, 205884.921875]",nw,Savage_Main,Tiramisu_0810,7,197740.390625,205884.921875,False


In [84]:
for i in range(len(drop_data)):
    df = drop_data[i]
    df = df.drop(columns= ["drop_loc_cat", "drop_loc_raw", "player", "rank"])
    drop_data[i] = df
drop_data[0]

,flight_path,map,x_drop_loc_raw,y_drop_loc_raw,isWinner
0,nw,Savage_Main,339081.093750,152563.390625,True
2,nw,Savage_Main,349186.906250,160740.125000,True
3,nw,Savage_Main,345526.093750,154371.609375,True
4,nw,Savage_Main,379695.093750,149027.328125,True
6,nw,Savage_Main,366499.625000,120417.921875,True
11,nw,Savage_Main,363794.187500,114268.429688,True
13,nw,Savage_Main,343474.406250,156387.343750,True
30,nw,Savage_Main,202218.906250,197842.250000,False
33,nw,Savage_Main,194840.250000,202073.265625,False
38,nw,Savage_Main,197740.390625,205884.921875,False


In [89]:
import sklearn
import sklearn.preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn.neighbors import KNeighborsClassifier
#old functions
def preprocess_data(df):
    labelencoder_X=sklearn.preprocessing.LabelEncoder()
    X = df.iloc[:,:].values
    df['flight_path'] = labelencoder_X.fit_transform(X[:, 1])
    df['map'] = labelencoder_X.fit_transform(X[:, 0])
    df['success_category'] = labelencoder_X.fit_transform(X[:, -1])
    return df
def train_model(df, max_k):
    try:
        X = df.iloc[:, :-1].copy()#assuming our target is at the very end of the dataframe
        y = df.iloc[:, -1].copy()
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= .2)
        x_train = sklearn.preprocessing.scale(x_train)
        x_test = sklearn.preprocessing.scale(x_test)
        k_scores = []
        for k in range(1,max_k):
            knn = KNeighborsClassifier(n_neighbors=k)
            cv_scores = cross_val_score(knn, x_train, y_train, cv=10, scoring='accuracy')
            k_scores.append(cv_scores.mean())

        max_score = max(k_scores)

        #     get the arg max of the score array to determine the optimal value for k
        k_opt = None
        for i in range(len(k_scores)):
            if k_scores[i] == max_score:
                k_opt = i + 1

        knn = KNeighborsClassifier(n_neighbors=k_opt)
        kf = KFold(n_splits=10)

        # split the training data into different validation folds and run a training loop on it

        kf.get_n_splits(x_train)
        training_scores = []


        knn.fit(x_train, y_train)#we get about -2.something places per map accuracy
        # testing_accuracy = (knn.predict(x_test) - y_test).mean()

        predictions = knn.predict(x_test)
        correct_predictions = predictions[predictions == y_test]

        print("TESTING ACCURACY: ", len(correct_predictions) / len(predictions))
        return knn
    except Exception:
        print("Unable to train model for the following data: ", df)

In [90]:
temp = preprocess_data(drop_data[0])
train_model(temp, 20)
temp

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.


TESTING ACCURACY:  0.5553745928338762


,flight_path,map,x_drop_loc_raw,y_drop_loc_raw,isWinner
0,0,0,339081.093750,152563.390625,1
2,0,0,349186.906250,160740.125000,1
3,0,0,345526.093750,154371.609375,1
4,0,0,379695.093750,149027.328125,1
6,0,0,366499.625000,120417.921875,1
11,0,0,363794.187500,114268.429688,1
13,0,0,343474.406250,156387.343750,1
30,0,0,202218.906250,197842.250000,0
33,0,0,194840.250000,202073.265625,0
38,0,0,197740.390625,205884.921875,0


In [91]:
for df in drop_data:
    train_model(preprocess_data(df), 20)
    
    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.


TESTING ACCURACY:  0.5423452768729642


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.


TESTING ACCURACY:  0.519134775374376


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.


...

TESTING ACCURACY:  0.6666666666666666


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.


TESTING ACCURACY:  0.5614035087719298


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.


TESTING ACCURACY:  0.4791666666666667


In [ ]:
drop_data = drop_data.dropna()
train_model(drop_data, 20)

In [ ]:
a = np.array(['a', 'b', 'f', 'd'])
b = np.array(['f', 'b', 'e', 'd'])
a[a == b]

In [ ]:
c = pd.DataFrame(np.array([[1,2,3],
                      [4,5,6],
                      [7,8,2]]), columns = ['a', 'b', 'c'])

c.loc[[1,0]]
for i in c.index:
    print(i)